In [82]:
# PyToch
import torch
torch.cuda.empty_cache()
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.models as models
import torchvision.transforms as t
import torchvision.datasets as datasets
from torch.amp import GradScaler, autocast  # Para entrenamiento con precision mixta, que acelerar el entrenamiento
# Librerias complentarias
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
from tqdm import tqdm
from sklearn.metrics import f1_score

# FUNCIONES PRINCIPALES

## Creacion del Modelo
Utilizamos el modelo ResNet18, ya que VGG16 innecesariamente complejo para este escenario. La gran mayoría de los modelos modernos obtienen un gran rendimiento en MNIST con bastante facilidad. Sin embargo, el modelo base no está preparado para la clasificación de 10 clases. Por ello, tomamos solo las capas convolucionales y adaptamos la clasificación al problema de los 10 dígitos:

- AvgPooling: Reduce la dimensionalidad y disminuye el número de parámetros.
- Flatten: Aplana los valores para poder realizar la clasificación.
- Capa Densa: Extrae las características restantes.
- ReLU: Función de activación de la capa de salida.
- Capa de Clasificación: Clasifica la salida en el número predicho.

In [83]:
def create_model(pretrained=True):
    # Cargar ResNet18 preentrenado
    resnet18 = models.resnet18(pretrained=pretrained)
    
    # Modificar la primera capa convolucional para aceptar imágenes en escala de grises (1 canal)
    old_layer = resnet18.conv1
    resnet18.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
    
    # Ajustar los pesos de la primera capa si se usa un modelo preentrenado
    if pretrained:
        with torch.no_grad():
            resnet18.conv1.weight.data = old_layer.weight.sum(dim=1, keepdim=True)
    
    # Modificar la capa fully connected final para clasificar 10 clases (dígitos del 0 al 9)
    resnet18.fc = nn.Linear(resnet18.fc.in_features, 10)
    
    return resnet18

# ENTRENAMIENTO
Entrenamos el modelo sin aplicar técnicas de regularización como Hold-Out o Cross-Validation, dado que MNIST es un dataset relativamente simple y como mencionamos antes estos modelos pueden lograr un alto rendimiento con implementaciones básicas. Para comprobar el progreso, almacenamos las estadísticas de cada epoch y batch en un diccionario, mostrando los resultados de cada epoch y cada 200 batches. Utilizamos Adam como optimizador con una tasa de aprendizaje de 1e-4 y CrossEntropy como función de pérdida, ya que es la más adecuada para la clasificación multiclase.

In [84]:
def train_model(model, dataloader, optimizer, criterion, num_epochs=5, patience=2):
	device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
	print(f"Usando dispositivo: {'GPU' if device == torch.device('cuda:0') else 'CPU'}")
	model.to(device)
	scaler = GradScaler()
 
	# Historial para métricas
	history = {
		'train_loss': [],         # Pérdida por batch
		'train_acc': [],          # Precisión por batch
		'epoch_train_loss': [],   # Pérdida promedio por época
		'epoch_train_acc': [],    # Precisión promedio por época
		'training_time': 0.0      # Tiempo total de entrenamiento
	}

	best_loss = np.inf  # Mejor pérdida (inicializada con infinito)
	epochs_no_improve = 0  # Contador de epochs sin mejora
 
	start_time = time.time()
	for epoch in range(num_epochs):
		print(f'Epoch {epoch+1}/{num_epochs}')
		print('-' * 10)
		
		# Modo entrenamiento
		model.train()
		current_loss = 0.0
		current_corrects = 0
		total_samples = 0
		
		for i, (inputs, labels) in enumerate(tqdm(dataloader, desc=f"Epoch {epoch+1}")):
			inputs = inputs.to(device)
			labels = labels.to(device)
			
			# Poner a cero los gradientes
			optimizer.zero_grad()
			
			# Forward
			if device.type == 'cuda':
				with autocast(device_type='cuda'): 
					outputs = model(inputs)
					loss = criterion(outputs, labels)  # Calcular pérdida
			else:
				outputs = model(inputs) 
				loss = criterion(outputs, labels)  # Calcular pérdida
			
			# Backward + optimize
			scaler.scale(loss).backward()  # Escalar gradientes
			scaler.step(optimizer)  # Actualizar pesos
			scaler.update()  # Actualizar el escalado
			
			# Pérdida y precisión por batch
			history['train_loss'].append(loss.item())
			
			# Calcular la precisión
			_, predicted = torch.max(outputs, 1)
			batch_acc = torch.sum(predicted == labels.data) / inputs.size(0)
			history['train_acc'].append(batch_acc.item())
			
			# Acumular estadísticas
			current_loss += loss.item() * inputs.size(0)  # Pérdida acumulada
			current_corrects += torch.sum(predicted == labels.data)  # Predicciones correctas
			total_samples += inputs.size(0)
			
			# Imprimir estadísticas cada 200 batches
			if i % 200 == 0:
				print(f"  Batch {i}: Loss: {loss.item():.4f}, Acc: {batch_acc.item():.4f}")
		
		# Estadísticas por época
		epoch_loss = current_loss / total_samples
		epoch_acc = current_corrects / total_samples
		
		# Guardar estadísticas por época
		history['epoch_train_loss'].append(epoch_loss)
		history['epoch_train_acc'].append(epoch_acc.item())
		
		print(f'  Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

		# Early stopping
		if epoch_loss < best_loss:
			best_loss = epoch_loss  # Actualizar la mejor pérdida
			epochs_no_improve = 0  # Reiniciar el contador
		else:
			epochs_no_improve += 1  # Incrementar el contador si no hay mejora
			print(f'  No improvement in loss for {epochs_no_improve} epochs.')
			
			# Detener el entrenamiento si no hay mejora
			if epochs_no_improve >= patience:
				print(f'  Early stopping triggered after {epoch+1} epochs.')
				break


	# Calcular tiempo total de entrenamiento
	time_elapsed = time.time() - start_time
	history['training_time'] = time_elapsed

	print(f'Entrenamiento completado en {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
	print(f'Precisión Final: {epoch_acc:.4f}')

	return model, history

# TESTEO
Implementamos un testeo basico usando como metrica de rendimiento la pricision en el conjunto de test de nuestro modelo.

In [85]:
def test_model(model, test_loader):
	device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
	model.to(device)  # Movemos el modelo a la GPU si está disponible
	model.eval()  # Ponemos el modelo en modo evaluación
    
	running_corrects = 0
	total_samples = 0
	test_accuracies = []
	all_preds = []
	all_labels = []
    
	with torch.no_grad():
		# for inputs, labels in tqdm(test_loader, desc="Evaluando"):
		for inputs, labels in (test_loader):
			inputs = inputs.to(device)
			labels = labels.to(device)

			outputs = model(inputs)
			_, preds = torch.max(outputs, 1)

			batch_acc = torch.sum(preds == labels.data) / inputs.size(0)
			test_accuracies.append(batch_acc.item())
            # Acumular predicciones y etiquetas para calcular el F1-score global, necesitamos las etiquetas y predicciones en la CPU
			all_preds.extend(preds.cpu().numpy())
			all_labels.extend(labels.cpu().numpy())
   
			running_corrects += torch.sum(preds == labels.data)
			total_samples += inputs.size(0)

	test_acc = running_corrects / total_samples
	print(f'Precisión en test: {test_acc:.4f}')
	f1 = f1_score(all_labels, all_preds, average='macro')
	print(f'F1 Score: {f1:.4f}')
	return test_acc.item(), test_accuracies, f1

# FUNCIONES COMPLEMENTARIAS

In [86]:
# Grafica las metricas de entrenamiento y test
def plot_metrics(history, test_acc):
	plt.figure(figsize=(12, 5))

	# Gráfico de pérdida
	plt.subplot(1, 2, 1)
	plt.plot(history['train_loss'], label='Train Loss')
	plt.title(f'Loss')
	plt.xlabel('Época')
	plt.ylabel('Pérdida')
	plt.legend()

	# Gráfico de precisión
	plt.subplot(1, 2, 2)
	plt.plot(history['train_acc'], label='Train Accuracy')
	plt.axhline(y=test_acc, color='r', linestyle='--', label=f'Test Accuracy: {test_acc:.4f}')
	plt.title(f'Accuracy')
	plt.xlabel('Época')
	plt.ylabel('Precisión')
	plt.legend()

	plt.tight_layout()
	plt.show()

In [87]:
# DataFrame para comparar resultados
def compare_results(histories, test_metrics):
    df = pd.DataFrame([
        {
            'Modelo': model,
            'Precisión Test (%)': f"{test_metrics[model]['accuracy'] * 100:.2f}%",
            'F1-Score': f"{test_metrics[model]['f1_score']:.4f}",
            'Tiempo Entrenamiento (s)': f"{histories[model]['training_time']:.1f}",
            'Tiempo Inferencia (s)': f"{test_metrics[model]['inference_time']:.1f}",
            'Pérdida Final': f"{histories[model]['epoch_train_loss'][-1]:.4f}"
        }
        for model in histories
    ])

    print("\nTabla Comparativa de Resultados:\n", df)
    return df

In [88]:
def run_experiments(train_loader_basic, train_loader_augment, test_loader):
	criterion = nn.CrossEntropyLoss()
	histories = {}
	test_metrics = {}
	models = {}

	# Modelos a probar
	configurations = [
		("PREENTRENADO SIN DA", True, train_loader_basic),
		("Preentrenado CON DA", True, train_loader_augment),
		("DESDE CERO SIN DA", False, train_loader_basic),
		("DESDE CERO CON DA", False, train_loader_augment)
	]

	for config_name, pretrained, dataloader in configurations:
		print(f"\n\n{config_name}")
		print("="*50)
		
		# Crear modelo
		model = create_model(pretrained=pretrained)
		optimizer = optim.Adam(model.parameters(), lr=1e-4)
		
		# Entrenar modelo
		model, history = train_model(model, dataloader, optimizer, criterion)		
		# Evaluar modelo
		test_acc, test_accuracies, f1 = test_model(model, test_loader)

		# Guardar resultados
		histories[config_name] = history
		test_metrics[config_name] = {
			'accuracy': test_acc,
			'f1_score': f1,
			'test_accuracies': test_accuracies
		}		
		models[config_name] = model

	# Graficos
	# plot_metrics(histories, test_acc)
	# Tabla comparativa
	compare_results(histories, test_metrics)

	return histories, test_metrics, models

# CARGA DE DATOS
Cargamos el conjunto de __MNIST__, separándolo en entrenamiento y test; además de un set de entrenamiento con __Data Augmentation (DA)__. Esta DA consiste en:
- Padding de 4 píxeles
- Transformaciones afines con centro invariante (Giros en la imagen donde el centro queda en el mismo sitio)
- Cambios en brillo y contraste

No se incluyen transformaciones como __flips__ ya que estos generarían ruido en el dataset. Un ejemplo claro de esto sería los números 9 y 6, si realizamos un flip vertical, estos números se intercambiarían y sus etiquetas pasarían a ser erróneas.
Además, los elementos del dataset se pasan a _Tensores_ para poder trabajar con ellos en PyTorch, y normalizamos para evitar que grandes diferencias de valores nos generen valores incorrectos.

In [89]:
# Definir rutas y parámetros
route = 'MNIST'
batch_size = 128

# Definir transformaciones básicas y con augmentation
mytransforms = t.Compose([
	t.ToTensor(),
    # Antes de usar Normalize, se debe de transformar a tensor
	t.Normalize((0.5,), (0.5,))
])

myaugmentation = t.Compose([
	t.Pad(4), # Añadir padding de 4 píxeles
	t.RandomAffine(degrees=45, translate=(0.2, 0.2), scale=(0.75, 1.25), shear=15), # Transformacion manteniendo el centro invariante
	t.ColorJitter(brightness=(0.2, 0.8), contrast=(0.2, 0.8)), # Cambios en brillo y contraste
	mytransforms
])

# Cargar datasets
train_basic = datasets.MNIST(route, train=True, download=False, transform=mytransforms)
train_da = datasets.MNIST(route, train=True, download=False, transform=myaugmentation)
test = datasets.MNIST(route, train=False, download=False, transform=mytransforms)

# Crear dataloaders
train_loader_basic = DataLoader(train_basic, batch_size=batch_size, shuffle=True, num_workers=8)
train_loader_da= DataLoader(train_da, batch_size=batch_size, shuffle=True, num_workers=8)
test_loader = DataLoader(test, batch_size=batch_size, shuffle=False, num_workers=8)

print("Train dataset size: ", len(train_basic))
print("Test dataset size: ", len(test))

Train dataset size:  60000
Test dataset size:  10000


# EJECUCION DE MODELOS

In [90]:
histories, test_metrics, models = run_experiments(train_loader_basic, train_loader_da, test_loader)

best_model_name = max(test_metrics, key=lambda x: test_metrics[x]['accuracy'])
best_accuracy = test_metrics[best_model_name]['accuracy']
best_f1 = test_metrics[best_model_name]['f1_score']
print(f"\nMejor modelo: {best_model_name} con precisión de {best_accuracy * 100:.2f}% y F1-score de {best_f1:.4f}")
print("\nAnálisis comparativo completado.")

/home/clown/2-semester/vision_individual/vision_env/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/clown/2-semester/vision_individual/vision_env/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)




PREENTRENADO SIN DA
Usando dispositivo: GPU
Epoch 1/15
----------


Epoch 1:   2%|▏         | 10/469 [00:00<00:16, 27.89it/s]

  Batch 0: Loss: 2.5250, Acc: 0.1328


Epoch 1:  46%|████▌     | 215/469 [00:02<00:03, 84.24it/s]

...

  Batch 400: Loss: 0.1525, Acc: 0.9453


Epoch 15: 100%|██████████| 469/469 [00:06<00:00, 68.73it/s]

  Train Loss: 0.1136 Acc: 0.9638
Entrenamiento completado en 1m 43s
Precisión Final: 0.9638


Precisión en test: 0.2752
F1 Score: 0.2053


KeyError: 'inference_time'